# Analyse HTML Files mit Beautifulsoup

Import

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import glob 
import numpy as np
import re

Create lists for df



In [16]:
app_names = []
company_names = []
date_registry = []
status = []
days_left = []
app_price = []

# Analyzing each HTML-File in directory html_scripts
base_dir = "html_scripts"
html_files = glob.glob(f"{base_dir}\\*.html")

# Parsing each html file for the informationen wanted (App Name, Company, Registry Date, Status, App Price). 
# The classes were searched on the Website with inspect function.
for file in html_files:
   with open(file, 'r', encoding='utf8') as html_file:
      html_string = html_file.read()
      soup = BeautifulSoup(html_string, 'html.parser')
      
      # Search the class, where the app name is displayed. If it exists, strip and replace every character in 
      # the class besides the app-name + append it to the list app_names
      app_name_search = soup.find(class_="entity-app__header__name entity-app__header__name--large")
      if app_name_search:
        app_name_search = app_name_search.text.replace("\n", "").strip()
        app_names.append(app_name_search)
      
      # Search the class, where the company name is displayed. Strip and replace every character or () in the class 
      # besides the company-name. Only the first item in the class is selected with [0] + append it to the list company_names
      company_name_results = soup.find_all(class_="entity-app__subheader")
      company_names.append(company_name_results[0].get_text().replace("\n", "").strip().split(",")[0] if company_name_results else "")

      # Search the class, where the registry date is displayed. Strip and replace every character in the class besides
      # the registry date. Append it to the list date_registry. 
      date_registry.append(soup.find_all("div", {"class":"entity-app-answers__questions__answer"})[0].text.replace("\n", "").strip())
    
      # There are 3 status for the DIGA "vorläufig aufgenommen", "dauerhaft aufgenommen" and an empty string "", thus tmp 
      # was used to search + strip/replace the status. If it was empty the status "Gestrichen" was added for the DIGA.
      tmp_status = soup.find(class_="entity-app__app-type").text.replace("\n", "").strip()
      if tmp_status:
        status.append(tmp_status)
      else:
        status.append("Gestrichen")

      # The class where the price is mentioned includes a lot of text. It was difficult to find the price for every DIGA, 
      # because there was no uniform style for currency ("€", "EUR", "Euro")
      app_price_results = soup.find_all("div", {"class":"entity-app-answers__questions__answer"})
      is_found = False
      for ap in app_price_results:
            app_price_text = ap.get_text()
            app_price_text = app_price_text.replace("\n", "").strip()
            #find number betwenn 0-9 with 2 or 3 digits, then a comma then again number bewteen ...
            matches_list = re.findall("([0-9]{2,3},[0-9]{0,2}) €", app_price_text)
            if matches_list:
              app_price.append(matches_list[0].replace(",", "."))
              is_found = True
              break
      # For every DIGA not using "€" for the price information "-1" is added to the list.
      if not is_found:     
        app_price.append(-1)


          
      


Number of files = 33 files


In [18]:
# Check Number of Status-Types in Stauts
unique_status = np.unique(status)
unique_status

array(['Dauerhaft aufgenommen', 'Gestrichen', 'Vorläufig aufgenommen'],
      dtype='<U21')

# Data Cleaning

1. Check if we have 33 data points for each Variable

In [19]:
print("Number of Names = ", len(app_names))
print("Number of Companies = ", len(company_names))
print("Number of Status = ", len(status))
print("Number of Dates = ", len(date_registry))
print("Number of Prices = ", len(app_price))

Number of Names =  33
Number of Companies =  33
Number of Status =  33
Number of Dates =  33
Number of Prices =  33


2. Check manualy for the missing price information and add them to the list.

In [24]:
# Create a directory to see which DIGA received a "-1"
dictionary = dict(zip(app_names, app_price))
dictionary





{'Selfapys Online-Kurs bei Generalisierter Angststörung': '540.00',
 'Selfapys Online-Kurs bei Panikstörung': '540.00',
 'NichtraucherHelden-App': '329.00',
 'Novego: Depressionen bewältigen': '249.00',
 'neolexon Aphasie': '487.90',
 'Kranus Edera': '656.88',
 'HelloBetter ratiopharm chronischer Schmerz': '599.00',
 'Cara Care für Reizdarm': '718.20',
 'HelloBetter Diabetes und Depression': '599.00',
 'Meine Tinnitus App - Das digitale Tinnitus Counseling': -1,
 'HelloBetter Vaginismus Plus': '599.00',
 'HelloBetter Panik': '599.00',
 'zanadio': '499.80',
 'Invirto- Die Therapie gegen Angst': -1,
 'M-sense Migräne': '219.98',
 'velibra': -1,
 'Mindable: Panikstörung und Agoraphobie': -1,
 'Kalmeda': '203.97',
 'Vivira': -1,
 'elevida': -1,
 'deprexis': -1,
 'somnio': -1,
 'Rehappy': -1,
 'Vitadio': '420.00',
 'vorvida': -1,
 'Oviva Direkt für Adipositas': -1,
 'Mika': -1,
 'Selfapys Online-Kurs bei Depression': -1,
 'ESYSTA App & Portal – Digitales Diabetesmanagement': '249.86',
 'CAN

In [26]:
# Review and Update prices for missing price information
app_price[2] = '329.00'
app_price[9] = '499.00'
app_price[13] = '620.00'
app_price[15] = '230.00'
app_price[16] = '576.00'
app_price[18] = '239.96'
app_price[19] = '243.00'
app_price[20] = '210.00'
app_price[21] = '224.99'
app_price[22] = '449.00'
app_price[24] = '476.00'
app_price[25] = '445.00'
app_price[26] = '499.00'
app_price[27] = '540.00'
app_price[30] = '599.00'


Create a dataframe for the data

In [32]:
df_diga = pd.DataFrame(data={"Name": app_names, "Unternehmen": company_names, "Status": status, "Aufnahmedatum": date_registry, "Preis": app_price})
df_diga

,Name,Unternehmen,Status,Aufnahmedatum,Preis
0,Selfapys Online-Kurs bei Generalisierter Angst...,Selfapy GmbH,Vorläufig aufgenommen,19.06.2021,540.00
1,Selfapys Online-Kurs bei Panikstörung,Selfapy GmbH,Vorläufig aufgenommen,19.06.2021,540.00
2,NichtraucherHelden-App,NichtraucherHelden GmbH,Vorläufig aufgenommen,03.07.2021,329.00
3,Novego: Depressionen bewältigen,IVPNetworks GmbH,Vorläufig aufgenommen,10.10.2021,249.00
4,neolexon Aphasie,Limedix GmbH,Vorläufig aufgenommen,06.02.2022,487.90
5,Kranus Edera,Kranus Health GmbH,Vorläufig aufgenommen,18.12.2021,656.88
6,HelloBetter ratiopharm chronischer Schmerz,GET.ON Institut für Online Gesundheitstraining...,Vorläufig aufgenommen,18.12.2021,599.00
7,Cara Care für Reizdarm,HiDoc Technologies GmbH,Vorläufig aufgenommen,26.12.2021,718.20
8,HelloBetter Diabetes und Depression,GET.ON Institut für Online Gesundheitstraining...,Dauerhaft aufgenommen,11.12.2021,599.00
9,Meine Tinnitus App - Das digitale Tinnitus Cou...,Sonormed GmbH,Vorläufig aufgenommen,06.03.2022,499.00


Check if there are Null Values within the Dataframe

In [33]:
df_diga.isnull().sum()

Name             0
Unternehmen      0
Status           0
Aufnahmedatum    0
Preis            0
dtype: int64

Save df as a CSV File without index

In [34]:
df_diga.to_csv('df_diga.csv', index=False)

In [2]:
df = pd.read_csv("df_diga.csv")
df

,Name,Unternehmen,Status,Aufnahmedatum,Preis
0,Selfapys Online-Kurs bei Generalisierter Angst...,Selfapy GmbH,Vorläufig aufgenommen,19.06.2021,540.00
1,Selfapys Online-Kurs bei Panikstörung,Selfapy GmbH,Vorläufig aufgenommen,19.06.2021,540.00
2,NichtraucherHelden-App,NichtraucherHelden GmbH,Vorläufig aufgenommen,03.07.2021,329.00
3,Novego: Depressionen bewältigen,IVPNetworks GmbH,Vorläufig aufgenommen,10.10.2021,249.00
4,neolexon Aphasie,Limedix GmbH,Vorläufig aufgenommen,06.02.2022,487.90
5,Kranus Edera,Kranus Health GmbH,Vorläufig aufgenommen,18.12.2021,656.88
6,HelloBetter ratiopharm chronischer Schmerz,GET.ON Institut für Online Gesundheitstraining...,Vorläufig aufgenommen,18.12.2021,599.00
7,Cara Care für Reizdarm,HiDoc Technologies GmbH,Vorläufig aufgenommen,26.12.2021,718.20
8,HelloBetter Diabetes und Depression,GET.ON Institut für Online Gesundheitstraining...,Dauerhaft aufgenommen,11.12.2021,599.00
9,Meine Tinnitus App - Das digitale Tinnitus Cou...,Sonormed GmbH,Vorläufig aufgenommen,06.03.2022,499.00


Check the data types

In [5]:
df.dtypes

Name              object
Unternehmen       object
Status            object
Aufnahmedatum     object
Preis            float64
dtype: object

Change dtype from column "Aufnahmedatum" to datetime64

In [3]:
import datetime as dt
df['Aufnahmedatum'] = pd.to_datetime(df['Aufnahmedatum'], dayfirst=True)
df.dtypes


Name                     object
Unternehmen              object
Status                   object
Aufnahmedatum    datetime64[ns]
Preis                   float64
dtype: object

Show first and last added DIGA

In [14]:
first_diga = df[df.Aufnahmedatum == df.Aufnahmedatum.min()]
last_diga = df[df.Aufnahmedatum == df.Aufnahmedatum.max()]


In [12]:
first_diga

,Name,Unternehmen,Status,Aufnahmedatum,Preis,max_date_12,max_date_24
15,velibra,GAIA AG,Dauerhaft aufgenommen,2020-01-10,230.0,2021-01-09,2022-01-09


In [15]:
last_diga

,Name,Unternehmen,Status,Aufnahmedatum,Preis,max_date_12,max_date_24
9,Meine Tinnitus App - Das digitale Tinnitus Cou...,Sonormed GmbH,Vorläufig aufgenommen,2022-06-03,499.0,2023-06-03,2024-06-02


Add a Column with days left in trial period

In [9]:
from datetime import timedelta
df["max_date_12"] = df["Aufnahmedatum"] + timedelta(days=365)
df["max_date_24"] = df["Aufnahmedatum"] + timedelta(days=365*2)
df.head()




,Name,Unternehmen,Status,Aufnahmedatum,Preis,max_date_12,max_date_24
0,Selfapys Online-Kurs bei Generalisierter Angst...,Selfapy GmbH,Vorläufig aufgenommen,2021-06-19,540.0,2022-06-19,2023-06-19
1,Selfapys Online-Kurs bei Panikstörung,Selfapy GmbH,Vorläufig aufgenommen,2021-06-19,540.0,2022-06-19,2023-06-19
2,NichtraucherHelden-App,NichtraucherHelden GmbH,Vorläufig aufgenommen,2021-03-07,329.0,2022-03-07,2023-03-07
3,Novego: Depressionen bewältigen,IVPNetworks GmbH,Vorläufig aufgenommen,2021-10-10,249.0,2022-10-10,2023-10-10
4,neolexon Aphasie,Limedix GmbH,Vorläufig aufgenommen,2022-06-02,487.9,2023-06-02,2024-06-01


Questions concerning ratio accepted permanently - deleted

In [16]:
status_permanent = df[df.Status == "Dauerhaft aufgenommen"]
len(status_permanent)
status_permanent

,Name,Unternehmen,Status,Aufnahmedatum,Preis,max_date_12,max_date_24
8,HelloBetter Diabetes und Depression,GET.ON Institut für Online Gesundheitstraining...,Dauerhaft aufgenommen,2021-11-12,599.00,2022-11-12,2023-11-12
10,HelloBetter Vaginismus Plus,GET.ON Institut für Online Gesundheitstraining...,Dauerhaft aufgenommen,2022-04-02,599.00,2023-04-02,2024-04-01
11,HelloBetter Panik,GET.ON Institut für Online Gesundheitstraining...,Dauerhaft aufgenommen,2022-03-04,599.00,2023-03-04,2024-03-03
15,velibra,GAIA AG,Dauerhaft aufgenommen,2020-01-10,230.00,2021-01-09,2022-01-09
17,Kalmeda,mynoise GmbH,Dauerhaft aufgenommen,2020-09-25,203.97,2021-09-25,2022-09-25
18,Vivira,Vivira Health Lab GmbH,Dauerhaft aufgenommen,2020-10-22,239.96,2021-10-22,2022-10-22
19,elevida,GAIA AG,Dauerhaft aufgenommen,2020-12-15,243.00,2021-12-15,2022-12-15
20,deprexis,GAIA AG,Dauerhaft aufgenommen,2021-02-20,210.00,2022-02-20,2023-02-20
21,somnio,mementor DE GmbH,Dauerhaft aufgenommen,2020-10-22,224.99,2021-10-22,2022-10-22
24,vorvida,GAIA AG,Dauerhaft aufgenommen,2021-06-05,476.00,2022-06-05,2023-06-05
